In [1]:
import jax
from brax.v1 import envs
from diversity_algorithms.controllers.fixed_structure_nn_flax import SimpleNeuralControllerFlax
import jax.numpy as jnp
from functools import partial


%load_ext autoreload
%autoreload 2

In [198]:
env = envs.create("ant")
key = jax.random.PRNGKey(0)

In [211]:
key

Array([0, 0], dtype=uint32)

In [199]:
import jax
import jax.numpy as jnp
random_key = jax.random.PRNGKey(0)
batch_size = 8192

In [ ]:
from diversity_algorithms.environments.brax_env import EvaluationFunctor
controller = SimpleNeuralControllerFlax(env.observation_size, env.action_size, n_hidden_layers=2, n_neurons_per_hidden=64)
EvaluationFunctor("ant", controller)

In [215]:
# Init population of controllers
controller = SimpleNeuralControllerFlax(env.observation_size, env.action_size, n_hidden_layers=2, n_neurons_per_hidden=64)
random_key, subkey = jax.random.split(random_key)
keys = jax.random.split(subkey, num=batch_size)
fake_batch = jnp.zeros(shape=(batch_size, env.observation_size))
init_variables = jax.vmap(controller.model.init)(keys, fake_batch).unfreeze()


In [216]:
step_fn = jax.vmap(jax.jit(env.step))
inference_fn = jax.vmap(jax.jit(controller.predict))
reset_fn = jax.vmap(jax.jit(env.reset))

In [217]:
@jax.jit
def eval(params, key):
	def eval_step(carry, _):
		states = carry
		actions = inference_fn(params, states.obs)
		next_states = step_fn(states, actions)
		return (next_states), (next_states.reward, next_states.metrics)

	keys = jax.random.split(key, num=batch_size)
	init_states = reset_fn(keys)
	return jax.lax.scan(eval_step, init_states, None, length=100)

In [219]:
final_states, (rewards, metrics) = eval(init_variables, key)

In [220]:
for i in range(20):
    final_states, (rewards, metrics) = eval(init_variables, key)

In [207]:
init_variables["params"]["Dense_0"]["kernel"].shape

(8192, 87, 64)

In [ ]:
pop_size = 100

# create inititial states
init_keys = jax.random.split(key, pop_size)
init_states = reset_fn(init_keys)

# get the shape of network parameters
shape = controller.get_parameters().shape

# create random parameters and convert them to a frozen dict
array_params = jax.vmap(lambda key: jax.random.uniform(key, shape))(init_keys)
params = jax.vmap(controller.array_to_fdict)(array_params)

In [ ]:
final_states, (rewards, metrics) = eval(init_states, params)